In [72]:
import pandas as pd
import pandas as pd
import geopandas as gpd
import shapely.geometry

In [73]:
# Leitura do arquivo csv gerado no processo anterior, com os acidentes de 2022
df_acidentes = pd.read_csv('df_acidentes_2022.csv')

# converte as colunas latitude e longitude para float
df_acidentes['latitude'] = df_acidentes['latitude'].str.replace(',', '.')
df_acidentes['latitude'] = pd.to_numeric(df_acidentes['latitude'])

df_acidentes['longitude'] = df_acidentes['longitude'].str.replace(',', '.')
df_acidentes['longitude'] = pd.to_numeric(df_acidentes['longitude'])

df_acidentes.head()

,Unnamed: 0,id,data_acidente,latitude,longitude,tipo_acidente
0,143600,8778739.0,2022-12-31,-23.493066,-46.805668,Não Fatal
1,143601,8778979.0,2022-12-31,-23.178480,-45.836573,Não Fatal
2,143602,8778023.0,2022-12-31,-22.386547,-47.553254,Não Fatal
3,143603,8779063.0,2022-12-31,-21.614215,-48.349298,Não Fatal
4,143604,8778531.0,2022-12-31,-23.500592,-46.794719,Não Fatal


In [74]:
# mapa com os acidentes de 2022 do infosiga
df_acidentes = gpd.GeoDataFrame(
    df_acidentes,
    geometry=gpd.points_from_xy(df_acidentes['longitude'], df_acidentes['latitude'],  crs="epsg:3857")
)

df_acidentes.head()
df_acidentes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 178731 entries, 0 to 178730
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Unnamed: 0     178731 non-null  int64   
 1   id             178731 non-null  float64 
 2   data_acidente  178731 non-null  object  
 3   latitude       164536 non-null  float64 
 4   longitude      164502 non-null  float64 
 5   tipo_acidente  178731 non-null  object  
 6   geometry       178731 non-null  geometry
dtypes: float64(3), geometry(1), int64(1), object(2)
memory usage: 9.5+ MB


In [108]:
# mapa com as zonas de tráfego da pesquisa OD 2017
file_path = 'Pesquisa_OD_2017_Zonas_de_Trafego_region.shp'
df_zonas = gpd.read_file(file_path)

#df_zonas.plot()

df_zonas['boundary'] = df_zonas.boundary
df_zonas['centroid'] = df_zonas.centroid
df_zonas['centroid.y'] = df_zonas.centroid.y
df_zonas['centroid.x'] = df_zonas.centroid.x

#df_zonas.set_crs('epsg:3857')
df_zonas


,geometry,boundary,centroid,centroid.y,centroid.x
0,"POLYGON Z ((333739.415 7394619.838 0.000, 3337...","LINESTRING Z (333739.415 7394619.838 0.000, 33...",POINT (333248.369 7394779.168),7.394779e+06,333248.369147
1,"POLYGON Z ((333106.146 7395425.480 0.000, 3331...","LINESTRING Z (333106.146 7395425.480 0.000, 33...",POINT (333667.401 7395305.522),7.395306e+06,333667.400883
2,"POLYGON Z ((333353.211 7393933.156 0.000, 3332...","LINESTRING Z (333353.211 7393933.156 0.000, 33...",POINT (333390.622 7394200.103),7.394200e+06,333390.622160
3,"POLYGON Z ((332742.619 7394795.328 0.000, 3327...","LINESTRING Z (332742.619 7394795.328 0.000, 33...",POINT (332449.199 7394473.104),7.394473e+06,332449.199127
4,"POLYGON Z ((332983.962 7395262.578 0.000, 3330...","LINESTRING Z (332983.962 7395262.578 0.000, 33...",POINT (332321.412 7395260.942),7.395261e+06,332321.412172
...,...,...,...,...,...
512,"POLYGON Z ((300336.368 7389696.621 0.000, 3003...","LINESTRING Z (300336.368 7389696.621 0.000, 30...",POINT (296718.125 7391240.594),7.391241e+06,296718.124920
513,"POLYGON Z ((303326.146 7397176.350 0.000, 3033...","LINESTRING Z (303326.146 7397176.350 0.000, 30...",POINT (303338.596 7394760.744),7.394761e+06,303338.596412
514,"POLYGON Z ((303292.377 7397174.790 0.000, 3033...","LINESTRING Z (303292.377 7397174.790 0.000, 30...",POINT (299315.725 7395169.706),7.395170e+06,299315.724559
515,"POLYGON Z ((296271.998 7398946.927 0.000, 2961...","LINESTRING Z (296271.998 7398946.927 0.000, 29...",POINT (304218.046 7405430.752),7.405431e+06,304218.046374


In [103]:
# Note the NaNs where the point did not intersect a boro
df_zonas = df_zonas.set_crs('epsg:3857')
df_acidentes = df_acidentes.set_crs('epsg:3857')
join_left_df = df_acidentes.sjoin(df_zonas, how="left", predicate="intersects")
join_left_df['index_right'].unique()

array([nan])

In [101]:
# Note the NaNs where the point did not intersect a boro
df_zonas = df_zonas.set_crs('epsg:3857')
df_acidentes = df_acidentes.set_crs('epsg:3857')
join_left_df = df_acidentes.sjoin(df_zonas, how="left", predicate="contains")
join_left_df['index_right'].unique()

array([nan])

In [100]:
# Note the NaNs where the point did not intersect a boro
df_zonas = df_zonas.set_crs('epsg:3857')
df_acidentes = df_acidentes.set_crs('epsg:3857')
join_left_df = df_acidentes.sjoin(df_zonas, how="left", predicate="within")
join_left_df['index_right'].unique()

array([nan])

In [99]:
# Note the NaNs where the point did not intersect a boro
df_zonas = df_zonas.set_crs('epsg:3857')
df_acidentes = df_acidentes.set_crs('epsg:3857')
join_left_df = df_acidentes.sjoin(df_zonas, how="left", predicate="touches")
join_left_df['index_right'].unique()

array([nan])

In [98]:
# Note the NaNs where the point did not intersect a boro
df_zonas = df_zonas.set_crs('epsg:3857')
df_acidentes = df_acidentes.set_crs('epsg:3857')
join_left_df = df_acidentes.sjoin(df_zonas, how="left", predicate="crosses")
join_left_df['index_right'].unique()

array([nan])

In [96]:
# Note the NaNs where the point did not intersect a boro
df_zonas = df_zonas.set_crs('epsg:3857')
df_acidentes = df_acidentes.set_crs('epsg:3857')
join_left_df = df_acidentes.sjoin(df_zonas, how="left", predicate="overlaps")
join_left_df['index_right'].unique()

array([nan])

In [79]:
df_acidentes.sjoin_nearest(df_zonas, how="left", distance_col="Distances")
# Note the optional Distances column with computed distances between each point
# and the nearest polydf geometry.

,Unnamed: 0,id,data_acidente,latitude,longitude,tipo_acidente,geometry,index_right,boundary,centroid,Distances
0,143600,8778739.0,2022-12-31,-23.493066,-46.805668,Não Fatal,POINT (-46.806 -23.493),474.0,"LINESTRING Z (316151.772 7351366.347 0.000, 31...",POINT (293996.446 7349265.973),7.343228e+06
1,143601,8778979.0,2022-12-31,-23.178480,-45.836573,Não Fatal,POINT (-45.837 -23.178),474.0,"LINESTRING Z (316151.772 7351366.347 0.000, 31...",POINT (293996.446 7349265.973),7.343227e+06
2,143602,8778023.0,2022-12-31,-22.386547,-47.553254,Não Fatal,POINT (-47.553 -22.387),474.0,"LINESTRING Z (316151.772 7351366.347 0.000, 31...",POINT (293996.446 7349265.973),7.343227e+06
3,143603,8779063.0,2022-12-31,-21.614215,-48.349298,Não Fatal,POINT (-48.349 -21.614),474.0,"LINESTRING Z (316151.772 7351366.347 0.000, 31...",POINT (293996.446 7349265.973),7.343226e+06
4,143604,8778531.0,2022-12-31,-23.500592,-46.794719,Não Fatal,POINT (-46.795 -23.501),474.0,"LINESTRING Z (316151.772 7351366.347 0.000, 31...",POINT (293996.446 7349265.973),7.343228e+06
...,...,...,...,...,...,...,...,...,...,...,...
178726,3710,3175555.0,2022-12-31 00:00:00,-23.661910,-46.751507,Fatal,POINT (-46.752 -23.662),474.0,"LINESTRING Z (316151.772 7351366.347 0.000, 31...",POINT (293996.446 7349265.973),7.343228e+06
178727,3711,3177906.0,2022-12-31 00:00:00,-23.406421,-46.768468,Fatal,POINT (-46.768 -23.406),474.0,"LINESTRING Z (316151.772 7351366.347 0.000, 31...",POINT (293996.446 7349265.973),7.343228e+06
178728,3712,3177610.0,2022-12-31 00:00:00,-23.502512,-46.617313,Fatal,POINT (-46.617 -23.503),474.0,"LINESTRING Z (316151.772 7351366.347 0.000, 31...",POINT (293996.446 7349265.973),7.343228e+06
178729,3713,3175556.0,2022-12-31 00:00:00,-23.442689,-46.304262,Fatal,POINT (-46.304 -23.443),474.0,"LINESTRING Z (316151.772 7351366.347 0.000, 31...",POINT (293996.446 7349265.973),7.343228e+06
